<a href="https://colab.research.google.com/github/anselmo-pitombeira/Notebooks/blob/master/corte_de_estoque_geracao_colunas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Solução do Problema de Corte de Estoque com Uso de Geração de Colunas

Neste notebook, implementamos um algoritmo de geração de colunas para o problema de corte de estoque com uso da biblioteca Pyomo.

##Instalação das bibliotecas necessárias

In [ ]:
!pip install pyomo
!apt-get install coinor-cbc

##Definições de funções

In [16]:
import numpy as np
import pyomo.environ as pyo

def pce_mestre(demanda, padroes):

    """
    Define um modelo de programação linear para o problema de corte de estoque,
    para ser usado como problema mestre restrito.

    inputs:
        demanda: Vetor com demandas por itens (vetor b)
        padroes: Matriz cujas colunas são os padrões de corte (matriz A)
    """

    model = pyo.ConcreteModel()       #Cria modelo
    model.N = np.shape(padroes)[1]    ##Número de colunas (padrões)
    model.m = np.shape(demanda)[0]    ##Número de itens
    
    #Variaveis de decisão do modelo
    model.x = pyo.Var([i for i in range(model.N)], domain=pyo.NonNegativeReals)   ##Note as variáveis reais não negativas 
    
    #Cria funcao objetivo
    def obj_fun(model):   
        return sum(model.x[j] for j in range(model.N))
            
    model.obj = pyo.Objective(rule=obj_fun, sense=pyo.minimize)
    
    ##Restrições
    def demand(model, i):
        return sum(model.x[j]*padroes[i][j] for j in range(model.N)) >= demanda[i]
    
    model.demand_res = pyo.Constraint(range(model.m), rule=demand)
 
    return model

def subproblema_mochila(c, p, L):
    """
    Constroi um modelo do subproblema da mochila.

    Inputs:
        c - Vetor de coeficientes associados a cada item (valores das variáveis duais obtidas do prob. mestre)
        p - Vetor com comprimentos de cada item
        L - Capacidade da mochila (comprimento do objeto em estoque)
    
    """
    n = c.size ##Número de itens
    modelo = pyo.ConcreteModel()
    modelo.x = pyo.Var([i for i in range(n)], domain=pyo.NonNegativeIntegers)                       ##Var. de decisão
    modelo.obj = pyo.Objective(expr=sum(c[i]*modelo.x[i] for i in range(n)), sense=pyo.maximize)    ##Função-objetivo
    modelo.res_capacidade = pyo.Constraint(expr=sum(p[i]*modelo.x[i] for i in range(n)) <=L)        ##Restrição de mochila
    
    return modelo

def solve_geracao_colunas(demanda, tam, L, maxiter=1000):
    
    m = len(tam)           ##Número de tamanhos possíveis dos itens
    padroes = np.eye(m)    ##Padroes iniciais (matriz identidade)
    
    ##Cria o solver (outras opções possíveis é cplex, gurobi ou glpk, se estiverem instalados)
    solver = pyo.SolverFactory('cbc', executable='/usr/bin/cbc')
    
    ##Loop principal
    for i in range(maxiter):
        print("Iteração = ",i)
        mestre = pce_mestre(demanda, padroes)                    ##Cria o modelo do problema mestre restrito
        mestre.dual = pyo.Suffix(direction=pyo.Suffix.IMPORT)    ##Definição das variáveis duais (preços-sombra)
        solver.solve(mestre)                                     ##Resolve o problema mestre restrito 
        print("Fobj mestre = ", mestre.obj.expr())
    
        ##Extração dos valores as variáveis duais
        duais = []
        for i in range(m):
            duais.append(mestre.dual[mestre.demand_res[i]])
        
        duais = np.array(duais)
        
        ##Resolve subproblema da mochilha
        subproblema = subproblema_mochila(duais, tam, L)
        solver.solve(subproblema)
        valor_fobj = subproblema.obj.expr()

        print("Valor subproblema = ", valor_fobj)

        ##Testa o valor da fobj do problema da mochila
        if valor_fobj <= 1.0:    
            print("Otimo atingido")
            break
        else:   ##Adiciona novo padrão
            novo_padrao = np.array(list(subproblema.x.get_values().values()))    ##Recupera novo padrão (solução ótima do problema da mochila)
            padroes = np.hstack([padroes, novo_padrao.reshape(-1,1)])            ##Adiciona novo padrão (adiciona coluna à matriz de padrões)
    
    solucao = np.array(list(mestre.x.get_values().values()))
    ##padroes_usados = padroes[:, solucao>0.0]
    ##solucao_arred = np.ceil(solucao[solucao>0.0])
    ##solucao_arred = np.ceil(solucao)
    
    return padroes, solucao


##Aplicação em um exemplo

In [ ]:
tamanhos = [200, 400, 600, 800, 1000]
demanda = [30, 60, 45, 30, 15]
L = 1200
padroes, solucao = solve_geracao_colunas(demanda, tamanhos, L, maxiter=1000)
solucao_arred = np.ceil(solucao)

print("\nPadroes finais = \n", padroes)
print("\nQuantidades de padroes usados = \n", solucao)
print("\nValor ótimo fobj (quant. de objetos cortados) = ", solucao.sum())
print("\nSolução arredondada = \n", solucao_arred)
print("\nNúmero de objetos cortados = \n", solucao_arred.sum())
print("\nQuantidades de itens produzidos =\n", padroes.dot(solucao_arred))
print("\nDemanda =",demanda)
